In [ ]:
#PyQt of 3 element filter with graph and sliders 

In [3]:
import sys
import cv2
import numpy as np
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QImage, QPixmap
import os

def normalize_and_dilate(img):
    img = np.nan_to_num(img)
    norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    norm_img = norm_img.astype(np.uint8)
    
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(norm_img, kernel, iterations=3)
    return norm_img, dilated  # return both for detection and display

def detect_blobs(img_norm, img_orig, min_thresh=100):
    params = cv2.SimpleBlobDetector_Params()
    params.minThreshold = min_thresh
    params.maxThreshold = 255
    params.filterByArea = True
    params.minArea = 200
    params.maxArea = 50000
    params.filterByCircularity = False
    params.filterByConvexity = False
    params.filterByInertia = False
    params.filterByColor = True
    params.blobColor = 255
    detector = cv2.SimpleBlobDetector_create(params)

    keypoints = detector.detect(img_norm)
    blobs = []
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        radius = int(kp.size / 2)
        mask = np.zeros(img_orig.shape, dtype=np.uint8)
        cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
        vals = img_orig[mask == 255]
        vals_dilated = img_norm[mask == 255]
        if vals.size > 0:
            blobs.append({
                'center': (x, y),
                'radius': radius,
                'size': kp.size,
                'max_intensity': vals.max(),
                'mean_intensity': vals.mean(),
                'mean_dilation': float(vals_dilated.mean())
            })
    return blobs

def draw_rectangles_on_image(image, blobs, color):
    """Draw rectangles on image at blob locations with given BGR color."""
    img_out = image.copy()
    for b in blobs:
        x, y = b['center']
        r = b['radius']
        top_left = (max(x - r, 0), max(y - r, 0))
        bottom_right = (min(x + r, img_out.shape[1]-1), min(y + r, img_out.shape[0]-1))
        cv2.rectangle(img_out, top_left, bottom_right, color, 2)
    return img_out

def process_and_show_rgb_with_rects(image_path_r, image_path_g, image_path_b):
    # Load original grayscale images (float or uint16/uint8)
    img_r = cv2.imread(image_path_r, cv2.IMREAD_UNCHANGED)
    img_g = cv2.imread(image_path_g, cv2.IMREAD_UNCHANGED)
    img_b = cv2.imread(image_path_b, cv2.IMREAD_UNCHANGED)

    # Normalize and dilate for detection
    img_r_norm, img_r_dilated = normalize_and_dilate(img_r)
    img_g_norm, img_g_dilated = normalize_and_dilate(img_g)
    img_b_norm, img_b_dilated = normalize_and_dilate(img_b)

    # Detect blobs on normalized images
    blobs_r = detect_blobs(img_r_norm, img_r, min_thresh=100)
    blobs_g = detect_blobs(img_g_norm, img_g, min_thresh=100)
    blobs_b = detect_blobs(img_b_norm, img_b, min_thresh=100)

    print(f"Red channel blobs detected: {len(blobs_r)}")
    print(f"Green channel blobs detected: {len(blobs_g)}")
    print(f"Blue channel blobs detected: {len(blobs_b)}")

    # Normalize original images to 0-255 uint8 for display (without dilation)
    def norm_to_uint8(img):
        img = np.nan_to_num(img)
        norm_img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        return norm_img.astype(np.uint8)

    img_r_disp = norm_to_uint8(img_r)
    img_g_disp = norm_to_uint8(img_g)
    img_b_disp = norm_to_uint8(img_b)

    # Merge original normalized images into RGB
    merged_img = cv2.merge([img_b_disp, img_g_disp, img_r_disp])  # OpenCV uses BGR order, so we swap R and B here

    # Draw rectangles for blobs on merged image in channel colors:
    # Red blobs: red rectangles
    merged_img = draw_rectangles_on_image(merged_img, blobs_r, (0, 0, 255))  # Red color (B,G,R)
    # Green blobs: green rectangles
    merged_img = draw_rectangles_on_image(merged_img, blobs_g, (0, 255, 0))
    # Blue blobs: blue rectangles
    merged_img = draw_rectangles_on_image(merged_img, blobs_b, (255, 0, 0))

    # Convert merged image (BGR) to RGB for Qt
    rgb_img = cv2.cvtColor(merged_img, cv2.COLOR_BGR2RGB)

    # Convert to QImage
    height, width, channel = rgb_img.shape
    bytes_per_line = 3 * width
    q_img = QImage(rgb_img.data, width, height, bytes_per_line, QImage.Format_RGB888)

    # PyQt display
    app = QApplication(sys.argv)
    window = QWidget()
    window.setWindowTitle("Merged Original TIFF with Blob Rectangles")

    layout = QVBoxLayout()
    label = QLabel()
    label.setPixmap(QPixmap.fromImage(q_img))
    layout.addWidget(label)

    window.setLayout(layout)
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    # Fix Qt plugin path (your previous fix)
    os.environ["QT_QPA_PLATFORM_PLUGIN_PATH"] = os.path.join(os.environ["CONDA_PREFIX"], "lib", "qt", "plugins")

    img_r_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Fe_merged.tiff"
    img_g_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Ca_merged.tiff"
    img_b_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_S_merged.tiff"

    process_and_show_rgb_with_rects(img_r_path, img_g_path, img_b_path)


Red channel blobs detected: 0
Green channel blobs detected: 0
Blue channel blobs detected: 0


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


SystemExit: 0

In [1]:
import os
import sys
import cv2
import numpy as np
import tifffile as tiff
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QColor, QPen
from PyQt5.QtCore import Qt

# === Set QT plugin path if needed (esp. for conda users) ===
os.environ["QT_QPA_PLATFORM_PLUGIN_PATH"] = os.path.join(os.environ["CONDA_PREFIX"], "lib", "qt", "plugins")

# === Image paths ===
img_r_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Fe_merged.tiff"
img_g_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_Ca_merged.tiff"
img_b_path = "/home/codingcarlos/Desktop/BNL SULI Summer 2025/4-UCM/mosaic_stitched_200um/mosaic_200_S_merged.tiff"

# === Helper: normalize and dilate ===
def normalize_and_dilate(img):
    img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
    norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    dilated = cv2.dilate(norm, np.ones((5, 5), np.uint8), iterations=3)
    return norm, dilated

# === Helper: blob detection ===
def detect_blobs(img_norm, img_orig, min_thresh):
    params = cv2.SimpleBlobDetector_Params()
    params.minThreshold = min_thresh
    params.maxThreshold = 255
    params.filterByArea = True
    params.minArea = 200
    params.maxArea = 50000
    params.filterByCircularity = False
    params.filterByConvexity = False
    params.filterByInertia = False
    params.filterByColor = True
    params.blobColor = 255
    detector = cv2.SimpleBlobDetector_create(params)

    keypoints = detector.detect(img_norm)
    blobs = []
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        radius = int(kp.size / 2)
        mask = np.zeros(img_orig.shape, dtype=np.uint8)
        cv2.rectangle(mask, (x - radius, y - radius), (x + radius, y + radius), 255, thickness=-1)
        vals = img_orig[mask == 255]
        vals_dilated = img_norm[mask == 255]
        if vals.size > 0:
            blobs.append({
                'center': (x, y),
                'radius': radius,
                'size': kp.size,
                'max_intensity': vals.max(),
                'mean_intensity': vals.mean(),
                'mean_dilation': float(vals_dilated.mean())
            })
    return blobs

# === Load images ===
img_r = tiff.imread(img_r_path).astype(np.float32)
img_g = tiff.imread(img_g_path).astype(np.float32)
img_b = tiff.imread(img_b_path).astype(np.float32)

# Normalize and dilate for detection
img_r_norm, img_r_dilated = normalize_and_dilate(img_r)
img_g_norm, img_g_dilated = normalize_and_dilate(img_g)
img_b_norm, img_b_dilated = normalize_and_dilate(img_b)

# Detect blobs on *dilated* images
blobs_r = detect_blobs(img_r_dilated, img_r, min_thresh=100)
blobs_g = detect_blobs(img_g_dilated, img_g, min_thresh=100)
blobs_b = detect_blobs(img_b_dilated, img_b, min_thresh=100)

# Normalize original images for display
img_r_disp = cv2.normalize(img_r, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
img_g_disp = cv2.normalize(img_g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
img_b_disp = cv2.normalize(img_b, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Merge to RGB
merged_rgb = cv2.merge([img_b_disp, img_g_disp, img_r_disp])

# === Convert to QImage for display ===
height, width, channel = merged_rgb.shape
bytes_per_line = 3 * width
q_img = QImage(merged_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)

# === Draw blob bounding boxes on QImage ===
q_img_painted = QImage(q_img)  # Make a copy
painter = QPainter(q_img_painted)
pen_r = QPen(QColor("red"), 2)
pen_g = QPen(QColor("green"), 2)
pen_b = QPen(QColor("blue"), 2)

for blob in blobs_r:
    x, y = blob['center']
    r = blob['radius']
    painter.setPen(pen_r)
    painter.drawRect(x - r, y - r, 2 * r, 2 * r)

for blob in blobs_g:
    x, y = blob['center']
    r = blob['radius']
    painter.setPen(pen_g)
    painter.drawRect(x - r, y - r, 2 * r, 2 * r)

for blob in blobs_b:
    x, y = blob['center']
    r = blob['radius']
    painter.setPen(pen_b)
    painter.drawRect(x - r, y - r, 2 * r, 2 * r)

painter.end()

# === PyQt5 GUI ===
app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Merged TIFF Image with Blob Bounding Boxes")

layout = QVBoxLayout()
label = QLabel()
label.setPixmap(QPixmap.fromImage(q_img_painted))
layout.addWidget(label)

window.setLayout(layout)
window.show()
sys.exit(app.exec_())


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


SystemExit: 0

/home/codingcarlos/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
^ colors are backwards and need to be swapped we have the red around blue boxes and blue around red boxes, green is fine 